## ola lyrics bot
#### Retweet quote tweets and reply on mention
Connect to Spotify and Genius to post lyrics on [Twitter](https://twitter.com/olalyricsbot)

In [1]:
import spotipy
import spotipy.oauth2 as oauth2
from spotipy.oauth2 import SpotifyOAuth
from spotipy.oauth2 import SpotifyClientCredentials
import lyricsgenius
import tweepy
import random
import os
from dotenv import load_dotenv

#### Spotify API Authorization

In [2]:
load_dotenv()

auth_manager = SpotifyClientCredentials(client_id=os.getenv("SPOTIPY_CLIENT_ID"), client_secret=os.getenv("SPOTIPY_CLIENT_SECRET"))
sp = spotipy.Spotify(auth_manager=auth_manager)

USERNAME = 'aleksandra.jedrych'
PLAYLIST_ID = '2VQxlEsq39DGjFxJ0o5nMo'

Download song ids from a dedicated playlist

In [3]:
def download_song_ids(username, playlist_id):
    song_ids = []
    playlist = sp.user_playlist(username, playlist_id)
    for item in playlist['tracks']['items']:
        song = item['track']
        song_ids.append(song['id'])
    return song_ids

songs_ids = download_song_ids(USERNAME, PLAYLIST_ID)
print(songs_ids)


['37ZJ0p5Jm13JPevGcx4SkF', '0GONea6G2XdnHWjNZd6zt3', '6Qyc6fS4DsZjB2mRW9DsQs', '7Ewz6bJ97vUqk5HdkvguFQ', '5t5rCnsgRBtcKqTB7SbD1Q', '3ovjw5HZZv43SxTwApooCM', '1pTGc8pwyo6xtgXBKCBcFn', '77NNZQSqzLNqh2A9JhLRkg', '5kK1Iru9ogP3Iy1zsANU1n', '5B8N5rPOmTVVGpuBMK2Vby', '60wwxj6Dd9NJlirf84wr2c', '390AWnOn2rfe9FzQjYmxIH', '6noSuYjYAXxM91DEaTAaFy', '2a3U3GTCvrcma7wetBtOpi', '1uzWOoJdADfstQuFtQFTUn', '3MrWxJaD2AT0W9DjWF64Vm', '7fpBtc7p3hFrEbDVc32ltY', '3EPXxR3ImUwfayaurPi3cm', '6N22FZs2ZhPBYi3b9XPajV', '6c9nkeetGbsoXa7d5E1wHa', '2EiGECydkS2M8OCcRHQZhT', '0eZBeB2xFIS65jQHerispi', '54b8qPFqYqIndfdxiLApea', '1PZDyestTVydtHpaIcfUJg', '7lPN2DXiMsVn7XUKtOW1CS', '4uGIJG1jYFonGc4LGp5uQL', '4vpeKl0vMGdAXpZiQB2Dtd', '39q7xibBdRboeMKUbZEB6g', '6xB7E0HOWznwiO0v56mqwD', '2qhASBzpbFhPRtrnZ5lLnz', '1zf8Xmj66XRiI6Etw2Ddu7', '0GD6Ug6ouPqkthlnT058aC', '6fqLce5TNAmNgbFwXD2DdG', '0I3cUqffwmH2mMUHIfpC0l', '5XcZRgJv3zMhTqCyESjQrF', '0bLRXQaWzmSXRXPmP6JnEF', '5lRzWDEe7UuedU2QPsFg0K', '0ow5yxh0ouMJ0om8EyEt8P', '6EkQmHBoux

Draw of a song id

In [4]:
id = songs_ids[random.randrange(len(songs_ids))]
print(id)

4JBNKQg27FoumUSo96r2pk


Collect data (title and main artist) needed to search forlyrics via the Genius API

In [5]:
def artist_genius(id):
    song_info = sp.track(id)
    artist = song_info['album']['artists'][0]['name'] #for genius (searching lyrics for tweet using title and main artist)
    return artist

In [7]:
artist_genius(id)

'Tom Odell'

In [6]:
def title(id):
    song_info = sp.track(id)
    title = song_info['name']
    return title

In [8]:
title(id)

'Magnetised'

Collect information about all authors to include in a tweet

In [9]:
def artists(id):
    song_info = sp.track(id)
    lista = []
    for i in range(0, len(song_info['album']['artists'])):
        lista.append(song_info['album']['artists'][i]['name']) #for tweet (including all artists performing the track)
    str_artists = ", "
    return(str_artists.join(lista))


In [10]:
artists(id)

'Tom Odell'

#### Connect to Genius API to download lyrics

In [11]:
genius = lyricsgenius.Genius(os.getenv("GENIUS_CLIENT_ID"))
genius.remove_section_headers = True

Download song lyrics

In [12]:
def raw_lyrics():
    lyrics = genius.search_song(title(id), artist_genius(id)).lyrics
    return lyrics

Create content of the tweet

In [13]:
def tweet_content(lyrics):
    lyrics = lyrics.split('\n')
    for index in range(len(lyrics)):
        if lyrics[index] == "" or "[" in lyrics[index]:
            lyrics[index] = "XXX"
    lyrics = [i for i in lyrics if i != "XXX"]

    random_num = random.randrange(0, len(lyrics)-1)
    tweet = lyrics[random_num] + "\n" + lyrics[random_num+1] + "\n" + "\n" + title(id) + " by " + artists(id)
    tweet = tweet.replace("\\", "")
    return tweet

#### Connect to Twitter API to post lyrics

In [14]:
api = tweepy.Client(bearer_token=os.getenv("BEARER_TOKEN"), consumer_key= os.getenv("CONSUMER_KEY"), consumer_secret=os.getenv("CONSUMER_SECRET"), access_token=os.getenv("ACCESS_TOKEN"), access_token_secret=os.getenv("ACCESS_TOKEN_SECRET") )

In [15]:
api.create_tweet(text=tweet_content(lyrics=raw_lyrics()))

Searching for "Magnetised" by Tom Odell...
Done.


Response(data={'id': '1474890670137651204', 'text': 'She keep me hanging on (Keep me hanging on)\nNorth to south, white to black\n\nMagnetised by Tom Odell'}, includes={}, errors=[], meta={})

In [16]:
import functions 
QUERY = "olalyricsbot"
MAX_RESULTS = 10

In [17]:
id = songs_ids[random.randrange(len(songs_ids))]
print(id)

5JQaG1VrTu4fbieUsDUdrt


In [57]:
def check(id):
    with open('last_tweet_ids.txt') as f:
        datafile = f.readlines()
    for line in datafile:
        if id in line:
            return True
    return False

In [69]:
def add_id_to_file(id):
    with open('last_tweet_ids.txt', 'w') as f:
        f.writelines(id)
        f.write('\n'.join(id))

In [70]:
functions.quote_tweets(QUERY, MAX_RESULTS)

[{'id': 1474897869454860293,
  'username': 'howareyouscully',
  'text': '@olalyricsbot ola draw lyrics pls'},
 {'id': 1474874021682495489,
  'username': 'olalyricsbot',
  'text': 'RT @langerspearl: najpiękniejsza piosenka o miłości'},
 {'id': 1474864114434551810,
  'username': 'olalyricsbot',
  'text': "RT @langerspearl: don't want to touch you but you're under my skin (deep in) https://t.co/98Gry9ESDJ"},
 {'id': 1474850453284429824,
  'username': 'langerspearl',
  'text': "don't want to touch you but you're under my skin (deep in) https://t.co/98Gry9ESDJ"},
 {'id': 1474801365247537168,
  'username': 'olalyricsbot',
  'text': 'RT @langerspearl: teraźniejsze piosenki już nie mają takiego klimatu...'},
 {'id': 1474784541571227650,
  'username': 'langerspearl',
  'text': 'teraźniejsze piosenki już nie mają takiego klimatu... https://t.co/UZBm0n77XT'}]

In [71]:
values_of_usernames = [dict['username'] for dict in functions.quote_tweets(QUERY, MAX_RESULTS)]
values_of_ids = [dict['id'] for dict in functions.quote_tweets(QUERY, MAX_RESULTS)]
values_of_text = [dict['text'] for dict in functions.quote_tweets(QUERY, MAX_RESULTS)]

In [72]:
for i in range(len(values_of_ids)):
    if check(str(values_of_ids[i])) == False:
        print("nie ma w tekscie")
        if values_of_usernames[i] != "olalyricsbot":
            print("nie ma olalyrics user")
            if "ola draw lyrics" in values_of_text[i] and check(str(values_of_ids[i])) == False:
                print("jestem tu")
                reply = "hello @" + values_of_usernames[i] + "\n" + "\n" + tweet_content(lyrics = raw_lyrics())
                api.like(values_of_ids[i])
                api.create_tweet(text=reply)
                print(values_of_ids[i], values_of_text[i], values_of_text[i])
                add_id_to_file(str(values_of_ids[i]))
                print("oto ja", str(values_of_ids[i])) 
            else:
                print("jestem teraz ru")
                api.like(values_of_ids[i])
                api.retweet(values_of_ids[i])
                print(values_of_ids[i], values_of_text[i], values_of_text[i])
                add_id_to_file(str(values_of_ids[i]))   
                print("teraz ja", str(values_of_ids[i])) 
        else: pass
    else: pass

nie ma w tekscie
nie ma olalyrics user
jestem tu
Searching for "Play the Game Tonight" by Kansas...
Done.
1474897869454860293 @olalyricsbot ola draw lyrics pls @olalyricsbot ola draw lyrics pls
oto ja 1474897869454860293
nie ma w tekscie
nie ma w tekscie
nie ma w tekscie
nie ma olalyrics user
jestem teraz ru
1474850453284429824 don't want to touch you but you're under my skin (deep in) https://t.co/98Gry9ESDJ don't want to touch you but you're under my skin (deep in) https://t.co/98Gry9ESDJ
teraz ja 1474850453284429824
nie ma w tekscie
nie ma w tekscie
nie ma olalyrics user
jestem teraz ru
1474784541571227650 teraźniejsze piosenki już nie mają takiego klimatu... https://t.co/UZBm0n77XT teraźniejsze piosenki już nie mają takiego klimatu... https://t.co/UZBm0n77XT
teraz ja 1474784541571227650


In [199]:
values_of_usernames = [dict['username'] for dict in quote_tweets(QUERY, MAX_RESULTS)]
values_of_ids = [dict['id'] for dict in quote_tweets(QUERY, MAX_RESULTS)]
values_of_text = [dict['text'] for dict in quote_tweets(QUERY, MAX_RESULTS)]

database_of_tweets = []
for i in range(len(values_of_usernames)):
    if values_of_usernames[i] == 'olalyricsbot' or values_of_ids[i] in database_of_tweets:
        print(values_of_usernames[i])
        for j in range(0, i):
            print(values_of_usernames[j])
            if 'ola draw lyrics' not in values_of_text[j]:
                database_of_tweets.append(values_of_ids[j])
                #api.like(values_of_ids[j])
                #api.retweet(values_of_ids[j])
            else:
                database_of_tweets.append(values_of_ids[j])
                reply = "hello @" + values_of_usernames[j] + "\n" + "\n" + tweet_content(lyrics = raw_lyrics())
                #api.like(values_of_ids[j])
                #api.create_tweet(text=reply)

print(database_of_tweets)
                

KeyboardInterrupt: 

In [200]:
values_of_usernames = [dict['username'] for dict in quote_tweets(QUERY, MAX_RESULTS)]
values_of_ids = [dict['id'] for dict in quote_tweets(QUERY, MAX_RESULTS)]
values_of_text = [dict['text'] for dict in quote_tweets(QUERY, MAX_RESULTS)]

In [201]:
database_of_tweets = []
print(database_of_tweets)

[]


In [202]:
database_of_tweets=[]
for i in range(len(values_of_ids)):
    if values_of_ids[i] not in database_of_tweets:
        if values_of_usernames[i] != "olalyricsbot":
            if "ola draw lyrics" in values_of_text[i]:
                reply = "hello @" + values_of_usernames[j] + "\n" + "\n" + tweet_content(lyrics = raw_lyrics())
                api.like(values_of_ids[i])
                api.create_tweet(text=reply)
                print(values_of_ids[i], values_of_text[i], values_of_text[i])
                database_of_tweets.append(values_of_ids[i])
            else:
                api.like(values_of_ids[i])
                api.retweet(values_of_ids[i])
                print(values_of_ids[i], values_of_text[i], values_of_text[i])
                database_of_tweets.append(values_of_ids[i])
        else: pass
    else: pass
print(database_of_tweets)

Searching for "Bat Out of Hell" by Meat Loaf...
Done.
1474873729020796937 @olalyricsbot hej ola draw lyrics @olalyricsbot hej ola draw lyrics
1474850453284429824 don't want to touch you but you're under my skin (deep in) https://t.co/98Gry9ESDJ don't want to touch you but you're under my skin (deep in) https://t.co/98Gry9ESDJ
1474784541571227650 teraźniejsze piosenki już nie mają takiego klimatu... https://t.co/UZBm0n77XT teraźniejsze piosenki już nie mają takiego klimatu... https://t.co/UZBm0n77XT
1472338642546966528 najpiękniejsza piosenka o miłości https://t.co/UoDTEd0IIc najpiękniejsza piosenka o miłości https://t.co/UoDTEd0IIc
[1474873729020796937, 1474850453284429824, 1474784541571227650, 1472338642546966528]


In [189]:
values_of_usernames = [dict['username'] for dict in functions.quote_tweets(QUERY, MAX_RESULTS)]
values_of_ids = [dict['id'] for dict in functions.quote_tweets(QUERY, MAX_RESULTS)]
values_of_text = [dict['text'] for dict in functions.quote_tweets(QUERY, MAX_RESULTS)]

database_of_tweets = []
for i in range(len(values_of_usernames)):
    if values_of_usernames[i] == 'olalyricsbot' or values_of_ids[i] in database_of_tweets:
        print(values_of_usernames[i])
        for j in range(0, i):
            print(values_of_usernames[j])
            if 'ola draw lyrics' not in values_of_text[j]:
                #api.like(values_of_ids[j])
                #api.retweet(values_of_ids[j])
                database_of_tweets.append(values_of_ids[j])
            else:
                reply = "hello @" + values_of_usernames[j] + "\n" + "\n" + tweet_content(lyrics = raw_lyrics())
                #api.like(values_of_ids[j])
               # api.create_tweet(text=reply)
                database_of_tweets.append(values_of_ids[j])

TypeError: quote_tweets() missing 1 required positional argument: 'max_results'

In [158]:
print(database_of_tweets)

[1474864114434551810, 1474850453284429824]
